In [ ]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
import scipy
from scipy.optimize import curve_fit
from scipy import interpolate
from matplotlib import pyplot as plt
from scipy import constants
from numpy.polynomial import Polynomial
import pandas as pd
import seaborn as sns

In [ ]:
#calculate some constants for HH

amu_to_au = 1822.89


mA_kg = 1.00784 * (10 ** (-3) / (6.022 * 10 ** 23) )
mB_kg = 24.305 * (10 ** (-3) / (6.022 * 10 ** 23) )
mA_au = 1.00784 * amu_to_au
mB_au = 24.305 * amu_to_au
mu_au = (mA_au * mB_au )/ (mA_au + mB_au)
mu_kg = (mA_kg * mB_kg) / (mA_kg + mB_kg)  
print("mu_au: ", mu_au)

In [ ]:
array_data = "/Users/proden/Code/SCQED-PCQED/array_data"
cavity_E_array_0_00_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0.npy").T [20:, :]
cavity_E_array_0_001_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_001.npy").T [20:, :]
cavity_E_array_0_005_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_005.npy").T[20:, :]
cavity_E_array_0_01_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_01.npy").T[20:, :]
cavity_E_array_0_02_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_02.npy").T[20:, :]
cavity_E_array_0_03_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_03.npy").T[20:, :]
cavity_E_array_0_04_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_04.npy").T[20:, :]
cavity_E_array_0_05_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_05.npy").T[20:, :]
r_data = np.load(array_data + "/MgH/fci_r_array_MgH.npy")[20:]
dipoles_0 = np.load(array_data + "/MgH/dipoles_MgH0.npy")[:,:,:,20:]
dipoles_001 = np.load(array_data + "/MgH/dipoles_MgH0_001.npy")[:,:,:,20:]
dipoles_005 = np.load(array_data + "/MgH/dipoles_MgH0_005.npy")[:,:,:,20:]
dipoles_01 = np.load(array_data + "/MgH/dipoles_MgH0_01.npy")[:,:,:,20:]
dipoles_02 = np.load(array_data + "/MgH/dipoles_MgH0_02.npy")[:,:,:,20:]
dipoles_03 = np.load(array_data + "/MgH/dipoles_MgH0_03.npy")[:,:,:,20:]
dipoles_04 = np.load(array_data + "/MgH/dipoles_MgH0_04.npy")[:,:,:,20:]
dipoles_05 = np.load(array_data + "/MgH/dipoles_MgH0_05.npy")[:,:,:,20:]
r_data = np.load(array_data + "/MgH/fci_r_array_MgH.npy")[20:]
N_R = r_data.shape[0]
N_R = r_data.shape[0]
#N_R = 200
r_data = np.linspace(0.5, 3.0, N_R)

## Calculation of k
Fit ground state PES of H2 to a quintic polynomial

In [ ]:
def declutter_E_array(E_array, r_data, discontinuity_threshold=0.0001, energy_diff_threshold = 0.02,  num_to_declutter = 2):
    E_array = np.copy(E_array)
    new_E_array = np.zeros_like(E_array)
    for i in range(0,num_to_declutter):
        previous_intersection = 0
        for z in range(0,50):
            for j in range(i+1, E_array.shape[1]):
                array1 = E_array[:, i]
                array2 = E_array[:, j]
                array1_from_previous_intersection = array1[previous_intersection:]
                array2_from_previous_intersection = array2[previous_intersection:]
                #find closest points
                closest_indices =np.where(np.abs(array1[previous_intersection:] - array2[previous_intersection:]) < energy_diff_threshold)
                if np.shape(closest_indices)[1] != 0:
                    #print(i)
                    #print(closest_indices)
                    pass
                try:
                    dy_1 = np.gradient(np.gradient(array1_from_previous_intersection, r_data[previous_intersection:], edge_order = 2), r_data[previous_intersection:], edge_order = 2)
                    idx_1 = np.where(abs(np.diff(dy_1)) >  discontinuity_threshold)[0]+2
                    dy_2= np.gradient(np.gradient(array2_from_previous_intersection, r_data[previous_intersection:], edge_order=2), r_data[previous_intersection:], edge_order=2)
                    idx_2 = np.where(abs(np.diff(dy_2)) > discontinuity_threshold)[0]+2
                    if (len(idx_1)!= 0 and len(idx_2) != 0 ):
                        mask_idx1_idx2 = np.isin(idx_1, idx_2)
                        indices_idx1_in_idx2 = np.where(mask_idx1_idx2)[0]
                        indices_idx1_in_idx2 = idx_1[indices_idx1_in_idx2]
                        #print(indices_idx1_in_idx2)
                        starting_index=0
                        for elem_index in range(len(indices_idx1_in_idx2)-1):
                            if indices_idx1_in_idx2[elem_index]+1 == indices_idx1_in_idx2[elem_index+1]:
                                starting_index = elem_index+1
                        indices_idx1_in_idx2 = indices_idx1_in_idx2[starting_index:]
                        if(len(indices_idx1_in_idx2) != 0 ):
                            mask_discontinuties_energydiff = np.isin(indices_idx1_in_idx2, closest_indices)
                            indices_discontinuties_in_energydiff = np.where(mask_discontinuties_energydiff)[0]
                            #print(indices_discontinuties_in_energydiff)
                            if len(indices_discontinuties_in_energydiff) != 0 :
                                idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[0]]+ previous_intersection
                                #print(idx)
                                array1_copy = np.array(array1, copy=True)
                                array1 = np.concatenate([array1[:idx],  array2[idx:]])
                                array2 =np.concatenate([array2[:idx] , array1_copy[idx:]])
                                E_array[:,i] = array1
                                E_array[:,j] = array2
                                previous_intersection = idx
                except():
                    print(i)
        new_E_array[:,i ] = E_array[:,i]
    return new_E_array

In [ ]:
import warnings
def declutter_E_array(E_array, dipoles,  r_data, discontinuity_threshold_std = 2, E_disc_threshold = 3, energy_diff_threshold_std = 2,  num_to_declutter = 2):
    
    E_array_copy = np.array(E_array, copy = True)

    d_matrices = np.zeros((num_to_declutter , num_to_declutter, r_data.shape[0]) )
    diff_d_matrices =  np.zeros((num_to_declutter , num_to_declutter, r_data.shape[0]-1))

    def build_dipole_mag_matrix(dipoles, n_elec, r_):

        
        d_matrix = np.zeros((n_elec,n_elec))

        def vector_magnitude(vector):
            return np.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)

        for i in range(n_elec):
            for j in range(n_elec):
                d_matrix[i][j] = vector_magnitude(dipoles[i,j,:,r_])


        #d_matrix = d_matrix + d_matrix.T - np.diag(np.diag(d_matrix))

        return d_matrix
    
    
    
    for i in range(0, r_data.shape[0] ):
        d_matrices[:, : , i] = build_dipole_mag_matrix(dipoles, num_to_declutter, i)

    diff_d_matrices = np.diff(np.diff(d_matrices))
    diff_d_matrices[diff_d_matrices < 10**-12] = 0



    #store discontinyutity locs as a list of lists like  
    # [   [loc, surface_1]  ,     [loc, surface_1] , [loc, surface_1] ]
    #find all discontinutiniutiy locations based on continutity of transition dipoles and dipoles
    discontinuity_locs = []


    for z in range(0, 1):
        previous_intersection = 0 
        for i in range(0,num_to_declutter): 

            for j in range(i, num_to_declutter):
                dipoles_diff = diff_d_matrices[i,j, previous_intersection:]
                mean = np.mean(diff_d_matrices[i, j, previous_intersection:])

                std = np.std(diff_d_matrices[i, j, previous_intersection:])

                discontinuity_threshold_pos =  mean + (std*discontinuity_threshold_std)
                discontinuity_threshold_neg =  mean - (std*discontinuity_threshold_std)
                idx_1 = np.sort(np.concatenate([ np.where( dipoles_diff >  discontinuity_threshold_pos )[0]+1 ,  np.where( dipoles_diff <  discontinuity_threshold_neg )[0]+1 ]))
                #print(idx_1)

                #plt.plot(diff_d_matrices[i, j, previous_intersection:])

                if len(idx_1) != 0:
                    #getting first point of crossover
                    if len(idx_1) > 1:
                        idx_1_copy = idx_1.copy()
                        while(len(idx_1_copy) > 0 ):
                            #print(idx_1_copy)
                            crossover_loc = None
                            for q in range(len(idx_1_copy) - 1):
                                #print(idx_1_copy)
                                if idx_1_copy[q] +1 == idx_1_copy[q+1]:
                                    #print("hello")
                                    idx_1_copy = idx_1_copy[q+1:]
                                    pass
                                else:
                                    crossover_loc = idx_1_copy[q]

                                    discontinuity_locs.append([crossover_loc, j])
                                    try:
                                        idx_1_copy = idx_1_copy[q+1:]
                                    except():
                                        idx_1_copy = []
                                        break

                                    break

                                if len(idx_1_copy == 1):
                                    break

                            if len(idx_1_copy) == 1:    
                                idx_1_copy = []
                                discontinuity_locs.append([idx_1[-1], j])
                                break                
                        
                    elif len(idx_1) == 1:
                        discontinuity_locs.append([idx_1[0], j])


    #sort and remove repaets
    discontinuity_locs  = [list(x) for x in set(tuple(x) for x in discontinuity_locs )]
    discontinuity_locs = sorted(discontinuity_locs, key=lambda x: x[0])
    #print(discontinuity_locs)

    #store discontinity locations like this
    # [   [loc, surface_1, surface_2]  ,     [loc, surface_1, surface_2] , [loc, surface_1, surface_2] ]
    #find all discontinutiniutiy locations based on continutity of transition dipoles and dipoles

    new_discontinuity_locs = []
    for q in range(len(discontinuity_locs)):
        loc1 = discontinuity_locs[q]
        for w in range(len(discontinuity_locs)):
            if w != q:
                loc2 = discontinuity_locs[w]

                if loc1[1] != loc2[1]:

                    if loc1[0] == loc2[0]:
                        new_discontinuity_locs.append([loc1[0] ,discontinuity_locs[q][1], discontinuity_locs[w][1]])

                    if loc1[0] == loc2[0] -1:
                        new_discontinuity_locs.append([loc2[0] ,discontinuity_locs[q][1], discontinuity_locs[w][1]])

                    if loc1[0] -1 == loc2[0]:
                        new_discontinuity_locs.append([loc1[0] ,discontinuity_locs[q][1], discontinuity_locs[w][1]])

    #sort and remove repeats again
    discontinuity_locs = new_discontinuity_locs
    # we need to remove one of these [loc, 1,2] and [loc, 2,1]
    for i in range(len(discontinuity_locs)):
        if discontinuity_locs[i][1] > discontinuity_locs[i][2]:

            copy = discontinuity_locs[i][2] 
            discontinuity_locs[i][2] = discontinuity_locs[i][1] 
            discontinuity_locs[i][1] = copy

    discontinuity_locs  = [list(x) for x in set(tuple(x) for x in discontinuity_locs )]
    discontinuity_locs = sorted(discontinuity_locs, key=lambda x: x[0])


    #for every discontinuity loc check and see if energies are very close using standard deviations of differences
    #crossover energy arrays

    crossover_points = []
    for i in range(len(discontinuity_locs)):
        loc = discontinuity_locs[i]

        array1 = E_array[:, loc[1]]
        array2 = E_array[:, loc[2]]


        #trying to determine how close two energy surfaces get, if they get very close this some evidence that they crossover
        diff_array1 = np.diff(array1 )
        diff_array2 = np.diff(array2 )
        std1 = np.std(np.abs(diff_array1))
        mean1 = np.mean(np.abs(diff_array1))
        std2 = np.std(np.abs(diff_array2))
        mean2 = np.mean(np.abs(diff_array2))
        energy_diff_threshold = ((mean1 + mean2)/2) + (((std1+std2)/2) * energy_diff_threshold_std)
        #find closest points
        #print(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])))
        closest_indices =np.where(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])) < energy_diff_threshold)[0]
        if loc[0] in closest_indices:

            crossover_points.append(loc)

        
    # a few points are like this :[1084, 3, 4] ,[1085, 3, 4], choose the larger one
    indices_to_pop = []
    for i in range(len(crossover_points)-1):
        if crossover_points[i][1] == crossover_points[i+1][1] and crossover_points[i][2] == crossover_points[i+1][2]:
            if crossover_points[i][0] +1 == crossover_points[i+1][0]:
                indices_to_pop.append(i)
    indices_to_pop.sort()
    indices_to_pop.reverse()
    for i in indices_to_pop:
        crossover_points.pop(i)


    #perform crossovers:
    for i in range(len(crossover_points)):


        idx = crossover_points[i][0] + 1

        array1 = E_array[:, crossover_points[i][1]]
        array2 = E_array[:, crossover_points[i][2]]
        

        array1_copy = np.array(array1, copy=True)
        array1 = np.concatenate([array1[:idx],  array2[idx:]])
        array2 =np.concatenate([array2[:idx] , array1_copy[idx:]])

        E_array[:,crossover_points[i][1]] = array1
        E_array[:,crossover_points[i][2]] = array2


        #go in and change crossover points to reflect changes in E_array
        surface_1 = crossover_points[i][1]
        surface_2 = crossover_points[i][2] 


        for p in range(len(crossover_points)):

            if crossover_points[p][1] == surface_1:
                crossover_points[p][1] = surface_2

            elif crossover_points[p][1] == surface_2:
                crossover_points[p][1] = surface_1

            if crossover_points[p][2] == surface_1:
                crossover_points[p][2] = surface_2

            elif crossover_points[p][2] == surface_2:
                crossover_points[p][2] = surface_1


    discontinuity_threshold_std = E_disc_threshold
    
    #dipole array orderd as [numroots][numroots][dipole vector][bondlength]
    E_array = np.copy(E_array)
    new_E_array = np.zeros_like(E_array)



    for i in range(0,num_to_declutter):
        previous_intersection = 0
        for z in range(0,50):
            for j in range(i+1, E_array.shape[1]):
                array1 = E_array[:, i]
                array2 = E_array[:, j]
                #Only want array from previous intersection so it doesnt get recrossed
                array1_from_previous_intersection = array1[previous_intersection:]
                array2_from_previous_intersection = array2[previous_intersection:]
                #trying to determine how close two energy surfaces get, if they get very close this some evidence that they crossover
                diff_array1 = np.diff(array1_from_previous_intersection)
                diff_array2 = np.diff(array2_from_previous_intersection)
                std1 = np.std(np.abs(diff_array1))
                mean1 = np.mean(np.abs(diff_array1))
                std2 = np.std(np.abs(diff_array2))
                mean2 = np.mean(np.abs(diff_array2))
                energy_diff_threshold = ((mean1 + mean2)/2) + (((std1+std2)/2) * energy_diff_threshold_std)
                #find closest points
                #print(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])))
                closest_indices =np.where(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])) < energy_diff_threshold)
                try:
                    #use discontinuties in second derivative, discontinutities defined using standard deviation
                    dy_1 = np.abs(np.gradient(np.gradient(array1_from_previous_intersection, r_data[previous_intersection:], edge_order = 1), r_data[previous_intersection:], edge_order = 1))
                    std = np.std(abs(np.diff(dy_1)))
                    mean = np.mean(abs(np.diff(dy_1)))
                    discontinuity_threshold =  mean + (std*discontinuity_threshold_std)
                    idx_1 = np.where(abs(np.diff(dy_1)) >  discontinuity_threshold)[0]+2
                    dy_2= np.abs(np.gradient(np.gradient(array2_from_previous_intersection, r_data[previous_intersection:], edge_order=1), r_data[previous_intersection:], edge_order=1))
                    std = np.std(abs(np.diff(dy_2)))
                    mean = np.mean(abs(np.diff(dy_2)))
                    discontinuity_threshold =  mean + (std*discontinuity_threshold_std)
                    idx_2 = np.where(abs(np.diff(dy_2)) > discontinuity_threshold)[0]+2
                    if (len(idx_1)!= 0 and len(idx_2) != 0 ):
                        mask_idx1_idx2 = np.isin(idx_1, idx_2)
                        indices_idx1_in_idx2 = np.where(mask_idx1_idx2)[0]
                        indices_idx1_in_idx2 = idx_1[indices_idx1_in_idx2]
                        # indices_idx1_in_idx2  = indices_idx1_in_idx2[ending_index:]
                        # starting_index=ending_index
                        # ending_index = starting_index
                        starting_index = 0
                        ending_index = 0
                        for elem_index in range(len(indices_idx1_in_idx2)-1):
                            #print("ayo: ", abs((indices_idx1_in_idx2[elem_index]) - (indices_idx1_in_idx2[elem_index+1])))
                            if abs((indices_idx1_in_idx2[elem_index]) - (indices_idx1_in_idx2[elem_index+1])) < 25 :
                                ending_index = ending_index+1
                            else:
                                break
                        indices_idx1_in_idx2 = indices_idx1_in_idx2[starting_index:ending_index]
                        if(len(indices_idx1_in_idx2) != 0 ):
                            mask_discontinuties_energydiff = np.isin(indices_idx1_in_idx2, closest_indices)
                            indices_discontinuties_in_energydiff = np.where(mask_discontinuties_energydiff)[0]
                            #print(indices_discontinuties_in_energydiff)
                            #print(indices_discontinuties_in_energydiff)
                            if len(indices_discontinuties_in_energydiff) != 0 :
                                for k in range(len(indices_discontinuties_in_energydiff) - 1):
                                    idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[k]]+ previous_intersection
                                    #print(idx)
                                    array1_copy = np.array(array1, copy=True)
                                    array1 = np.concatenate([array1[:idx],  array2[idx:idx+1], array1[idx+1:]])
                                    array2 = np.concatenate([array2[:idx] , array1_copy[idx:idx+1], array2[idx+1:]])
                                    E_array[:,i] = array1
                                    E_array[:,j] = array2
                                idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[-1]]+ previous_intersection
                                #print(idx)
                                array1_copy = np.array(array1, copy=True)
                                array1 = np.concatenate([array1[:idx],  array2[idx:]])
                                array2 =np.concatenate([array2[:idx] , array1_copy[idx:]])
                                #print(indices_idx1_in_idx2)
                                fitting_distance=10
                                if abs(indices_idx1_in_idx2[-1] - indices_idx1_in_idx2[0]) < fitting_distance:
                                    array1 = array1.tolist()
                                    array2 = array2.tolist()
                                    r_data_list = r_data.tolist()
                                    #fitting region
                                    end_discontinuity = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[-1]]+ previous_intersection
                                    start_discontinuity = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[0]]+ previous_intersection
                                    fit_E_data_end = min(end_discontinuity+fitting_distance, len(array1))
                                    fit_E_data_start= max(start_discontinuity-fitting_distance, 0)
                                    # print(fit_E_data_start)
                                    # print(fit_E_data_end)
                                    fitting_E_data = array1[fit_E_data_start: start_discontinuity] + array1[end_discontinuity: fit_E_data_end]
                                    fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
                                    with warnings.catch_warnings():
                                        warnings.simplefilter("ignore")
                                        poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
                                    r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
                                    polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
                                    array1 = array1[:fit_E_data_start] + polyvals.tolist() + array1[fit_E_data_end:]
                                    fitting_E_data = array2[fit_E_data_start: start_discontinuity] + array2[end_discontinuity: fit_E_data_end]
                                    fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
                                    with warnings.catch_warnings():
                                        warnings.simplefilter("ignore")
                                        poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
                                    r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
                                    polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
                                    array2 = array2[:fit_E_data_start] + polyvals.tolist() + array2[fit_E_data_end:]
                                E_array[:,i] = array1
                                E_array[:,j] = array2
                                previous_intersection = idx
                except():
                    print("uh oh")
        new_E_array[:,i ] = E_array[:,i]
    




    return E_array

In [ ]:
import warnings
def declutter_E_array(E_array, dipoles,  r_data, discontinuity_threshold_std = 2, energy_diff_threshold_std_p1 = 1,  E_disc_threshold = 3, energy_diff_threshold_std = 2,  num_to_declutter = 2):
    
    E_array_copy = np.array(E_array, copy = True)

    d_matrices = np.zeros((num_to_declutter , num_to_declutter, r_data.shape[0]) )
    diff_d_matrices =  np.zeros((num_to_declutter , num_to_declutter, r_data.shape[0]-1))

    def build_dipole_mag_matrix(dipoles, n_elec, r_):

        
        d_matrix = np.zeros((n_elec,n_elec))

        def vector_magnitude(vector):
            return np.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)

        for i in range(n_elec):
            for j in range(n_elec):
                d_matrix[i][j] = vector_magnitude(dipoles[i,j,:,r_])


        #d_matrix = d_matrix + d_matrix.T - np.diag(np.diag(d_matrix))

        return d_matrix
    
    
    
    for i in range(0, r_data.shape[0] ):
        d_matrices[:, : , i] = build_dipole_mag_matrix(dipoles, num_to_declutter, i)

    diff_d_matrices = np.diff(np.diff(d_matrices))
    diff_d_matrices[diff_d_matrices < 10**-12] = 0



    #store discontinyutity locs as a list of lists like  
    # [   [loc, surface_1]  ,     [loc, surface_1] , [loc, surface_1] ]
    #find all discontinutiniutiy locations based on continutity of transition dipoles and dipoles
    discontinuity_locs = []


    for z in range(0, 1):
        previous_intersection = 0 
        for i in range(0,num_to_declutter): 

            for j in range(i, num_to_declutter):
                dipoles_diff = diff_d_matrices[i,j, previous_intersection:]
                mean = np.mean(diff_d_matrices[i, j, previous_intersection:])

                std = np.std(diff_d_matrices[i, j, previous_intersection:])

                discontinuity_threshold_pos =  mean + (std*discontinuity_threshold_std)
                discontinuity_threshold_neg =  mean - (std*discontinuity_threshold_std)
                idx_1 = np.sort(np.concatenate([ np.where( dipoles_diff >  discontinuity_threshold_pos )[0]+1 ,  np.where( dipoles_diff <  discontinuity_threshold_neg )[0]+1 ]))
                #print(idx_1)

                #plt.plot(diff_d_matrices[i, j, previous_intersection:])

                if len(idx_1) != 0:
                    #getting first point of crossover
                    if len(idx_1) > 1:
                        idx_1_copy = idx_1.copy()
                        while(len(idx_1_copy) > 0 ):
                            #print(idx_1_copy)
                            crossover_loc = None
                            for q in range(len(idx_1_copy) - 1):
                                #print(idx_1_copy)
                                if idx_1_copy[q] +1 == idx_1_copy[q+1]:
                                    #print("hello")
                                    idx_1_copy = idx_1_copy[q+1:]
                                    pass
                                else:
                                    crossover_loc = idx_1_copy[q]

                                    discontinuity_locs.append([crossover_loc, j])
                                    try:
                                        idx_1_copy = idx_1_copy[q+1:]
                                    except():
                                        idx_1_copy = []
                                        break

                                    break

                                if len(idx_1_copy == 1):
                                    break

                            if len(idx_1_copy) == 1:    
                                idx_1_copy = []
                                discontinuity_locs.append([idx_1[-1], j])
                                break                
                        
                    elif len(idx_1) == 1:
                        discontinuity_locs.append([idx_1[0], j])


    #sort and remove repaets
    discontinuity_locs  = [list(x) for x in set(tuple(x) for x in discontinuity_locs )]
    discontinuity_locs = sorted(discontinuity_locs, key=lambda x: x[0])
    #print(discontinuity_locs)

    #store discontinity locations like this
    # [   [loc, surface_1, surface_2]  ,     [loc, surface_1, surface_2] , [loc, surface_1, surface_2] ]
    #find all discontinutiniutiy locations based on continutity of transition dipoles and dipoles

    new_discontinuity_locs = []
    for q in range(len(discontinuity_locs)):
        loc1 = discontinuity_locs[q]
        for w in range(len(discontinuity_locs)):
            if w != q:
                loc2 = discontinuity_locs[w]

                if loc1[1] != loc2[1]:

                    if loc1[0] == loc2[0]:
                        new_discontinuity_locs.append([loc1[0] ,discontinuity_locs[q][1], discontinuity_locs[w][1]])

                    if loc1[0] == loc2[0] -1:
                        new_discontinuity_locs.append([loc2[0] ,discontinuity_locs[q][1], discontinuity_locs[w][1]])

                    if loc1[0] -1 == loc2[0]:
                        new_discontinuity_locs.append([loc1[0] ,discontinuity_locs[q][1], discontinuity_locs[w][1]])

    #sort and remove repeats again
    discontinuity_locs = new_discontinuity_locs
    # we need to remove one of these [loc, 1,2] and [loc, 2,1]
    for i in range(len(discontinuity_locs)):
        if discontinuity_locs[i][1] > discontinuity_locs[i][2]:

            copy = discontinuity_locs[i][2] 
            discontinuity_locs[i][2] = discontinuity_locs[i][1] 
            discontinuity_locs[i][1] = copy

    discontinuity_locs  = [list(x) for x in set(tuple(x) for x in discontinuity_locs )]
    discontinuity_locs = sorted(discontinuity_locs, key=lambda x: x[0])


    #for every discontinuity loc check and see if energies are very close using standard deviations of differences
    #crossover energy arrays
    
    crossover_points = []
    for i in range(len(discontinuity_locs)):
        loc = discontinuity_locs[i]

        array1 = E_array[:, loc[1]]
        array2 = E_array[:, loc[2]]


        #trying to determine how close two energy surfaces get, if they get very close this some evidence that they crossover
        diff_array1 = np.diff(array1 )
        diff_array2 = np.diff(array2 )
        std1 = np.std(np.abs(diff_array1))
        mean1 = np.mean(np.abs(diff_array1))
        std2 = np.std(np.abs(diff_array2))
        mean2 = np.mean(np.abs(diff_array2))
        energy_diff_threshold = ((mean1 + mean2)/2) + (((std1+std2)/2) * energy_diff_threshold_std_p1)
        #find closest points
        #print(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])))
        closest_indices =np.where(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])) < energy_diff_threshold)[0]
        if loc[0] in closest_indices:

            crossover_points.append(loc)

        
    # a few points are like this :[1084, 3, 4] ,[1085, 3, 4], choose the larger one
    indices_to_pop = []
    for i in range(len(crossover_points)-1):
        if crossover_points[i][1] == crossover_points[i+1][1] and crossover_points[i][2] == crossover_points[i+1][2]:
            if crossover_points[i][0] +1 == crossover_points[i+1][0]:
                indices_to_pop.append(i)
    indices_to_pop.sort()
    indices_to_pop.reverse()
    for i in indices_to_pop:
        crossover_points.pop(i)


    #perform crossovers:
    for i in range(len(crossover_points)):


        idx = crossover_points[i][0] + 1

        array1 = E_array[:, crossover_points[i][1]]
        array2 = E_array[:, crossover_points[i][2]]


        

        array1_copy = np.array(array1, copy=True)
        array1 = np.concatenate([array1[:idx],  array2[idx:]])
        array2 =np.concatenate([array2[:idx] , array1_copy[idx:]])

        fitting_distance = 5
        array1 = array1.tolist()
        array2 = array2.tolist()
        r_data_list = r_data.tolist()
        #fitting region
        end_discontinuity = crossover_points[i][0]+1
        start_discontinuity = crossover_points[i][0] -1
        fit_E_data_end = min(end_discontinuity+fitting_distance, len(array1))
        fit_E_data_start= max(start_discontinuity-fitting_distance, 0)
        # print(fit_E_data_start)
        # print(fit_E_data_end)
        fitting_E_data = array1[fit_E_data_start: start_discontinuity] + array1[end_discontinuity: fit_E_data_end]
        fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
        r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
        polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
        array1 = array1[:fit_E_data_start] + polyvals.tolist() + array1[fit_E_data_end:]
        fitting_E_data = array2[fit_E_data_start: start_discontinuity] + array2[end_discontinuity: fit_E_data_end]
        fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
        r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
        polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
        array2 = array2[:fit_E_data_start] + polyvals.tolist() + array2[fit_E_data_end:]


        E_array[:,crossover_points[i][1]] = array1
        E_array[:,crossover_points[i][2]] = array2


        #go in and change crossover points to reflect changes in E_array
        surface_1 = crossover_points[i][1]
        surface_2 = crossover_points[i][2] 




        for p in range(len(crossover_points)):

            if crossover_points[p][1] == surface_1:
                crossover_points[p][1] = surface_2

            elif crossover_points[p][1] == surface_2:
                crossover_points[p][1] = surface_1

            if crossover_points[p][2] == surface_1:
                crossover_points[p][2] = surface_2

            elif crossover_points[p][2] == surface_2:
                crossover_points[p][2] = surface_1


    discontinuity_threshold_std = E_disc_threshold
    
    #dipole array orderd as [numroots][numroots][dipole vector][bondlength]
    E_array = np.copy(E_array)
    new_E_array = np.zeros_like(E_array)



    for i in range(0,num_to_declutter):
        previous_intersection = 0
        for z in range(0,50):
            for j in range(i+1, E_array.shape[1]):
                array1 = E_array[:, i]
                array2 = E_array[:, j]
                #Only want array from previous intersection so it doesnt get recrossed
                array1_from_previous_intersection = array1[previous_intersection:]
                array2_from_previous_intersection = array2[previous_intersection:]
                #trying to determine how close two energy surfaces get, if they get very close this some evidence that they crossover
                diff_array1 = np.diff(array1_from_previous_intersection)
                diff_array2 = np.diff(array2_from_previous_intersection)
                std1 = np.std(np.abs(diff_array1))
                mean1 = np.mean(np.abs(diff_array1))
                std2 = np.std(np.abs(diff_array2))
                mean2 = np.mean(np.abs(diff_array2))
                energy_diff_threshold = ((mean1 + mean2)/2) + (((std1+std2)/2) * energy_diff_threshold_std)
                #find closest points
                #print(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])))
                closest_indices =np.where(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])) < energy_diff_threshold)
                try:
                    #use discontinuties in second derivative, discontinutities defined using standard deviation
                    dy_1 = np.abs(np.gradient(np.gradient(array1_from_previous_intersection, r_data[previous_intersection:], edge_order = 1), r_data[previous_intersection:], edge_order = 1))
                    std = np.std(abs(np.diff(dy_1)))
                    mean = np.mean(abs(np.diff(dy_1)))
                    discontinuity_threshold =  mean + (std*discontinuity_threshold_std)
                    idx_1 = np.where(abs(np.diff(dy_1)) >  discontinuity_threshold)[0]+2
                    dy_2= np.abs(np.gradient(np.gradient(array2_from_previous_intersection, r_data[previous_intersection:], edge_order=1), r_data[previous_intersection:], edge_order=1))
                    std = np.std(abs(np.diff(dy_2)))
                    mean = np.mean(abs(np.diff(dy_2)))
                    discontinuity_threshold =  mean + (std*discontinuity_threshold_std)
                    idx_2 = np.where(abs(np.diff(dy_2)) > discontinuity_threshold)[0]+2
                    if (len(idx_1)!= 0 and len(idx_2) != 0 ):
                        mask_idx1_idx2 = np.isin(idx_1, idx_2)
                        indices_idx1_in_idx2 = np.where(mask_idx1_idx2)[0]
                        indices_idx1_in_idx2 = idx_1[indices_idx1_in_idx2]
                        # indices_idx1_in_idx2  = indices_idx1_in_idx2[ending_index:]
                        # starting_index=ending_index
                        # ending_index = starting_index
                        starting_index = 0
                        ending_index = 0
                        for elem_index in range(len(indices_idx1_in_idx2)-1):
                            #print("ayo: ", abs((indices_idx1_in_idx2[elem_index]) - (indices_idx1_in_idx2[elem_index+1])))
                            if abs((indices_idx1_in_idx2[elem_index]) - (indices_idx1_in_idx2[elem_index+1])) < 25 :
                                ending_index = ending_index+1
                            else:
                                break
                        indices_idx1_in_idx2 = indices_idx1_in_idx2[starting_index:ending_index]
                        if(len(indices_idx1_in_idx2) != 0 ):
                            mask_discontinuties_energydiff = np.isin(indices_idx1_in_idx2, closest_indices)
                            indices_discontinuties_in_energydiff = np.where(mask_discontinuties_energydiff)[0]
                            #print(indices_discontinuties_in_energydiff)
                            #print(indices_discontinuties_in_energydiff)
                            if len(indices_discontinuties_in_energydiff) != 0 :
                                for k in range(len(indices_discontinuties_in_energydiff) - 1):
                                    idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[k]]+ previous_intersection
                                    #print(idx)
                                    array1_copy = np.array(array1, copy=True)
                                    array1 = np.concatenate([array1[:idx],  array2[idx:idx+1], array1[idx+1:]])
                                    array2 = np.concatenate([array2[:idx] , array1_copy[idx:idx+1], array2[idx+1:]])
                                    E_array[:,i] = array1
                                    E_array[:,j] = array2
                                idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[-1]]+ previous_intersection
                                #print(idx)
                                array1_copy = np.array(array1, copy=True)
                                array1 = np.concatenate([array1[:idx],  array2[idx:]])
                                array2 =np.concatenate([array2[:idx] , array1_copy[idx:]])
                                #print(indices_idx1_in_idx2)
                                fitting_distance=10
                                if abs(indices_idx1_in_idx2[-1] - indices_idx1_in_idx2[0]) < fitting_distance:
                                    array1 = array1.tolist()
                                    array2 = array2.tolist()
                                    r_data_list = r_data.tolist()
                                    #fitting region
                                    end_discontinuity = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[-1]]+ previous_intersection
                                    start_discontinuity = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[0]]+ previous_intersection
                                    fit_E_data_end = min(end_discontinuity+fitting_distance, len(array1))
                                    fit_E_data_start= max(start_discontinuity-fitting_distance, 0)
                                    # print(fit_E_data_start)
                                    # print(fit_E_data_end)
                                    fitting_E_data = array1[fit_E_data_start: start_discontinuity] + array1[end_discontinuity: fit_E_data_end]
                                    fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
                                    with warnings.catch_warnings():
                                        warnings.simplefilter("ignore")
                                        poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
                                    r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
                                    polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
                                    array1 = array1[:fit_E_data_start] + polyvals.tolist() + array1[fit_E_data_end:]
                                    fitting_E_data = array2[fit_E_data_start: start_discontinuity] + array2[end_discontinuity: fit_E_data_end]
                                    fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
                                    with warnings.catch_warnings():
                                        warnings.simplefilter("ignore")
                                        poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
                                    r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
                                    polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
                                    array2 = array2[:fit_E_data_start] + polyvals.tolist() + array2[fit_E_data_end:]
                                E_array[:,i] = array1
                                E_array[:,j] = array2
                                previous_intersection = idx
                except():
                    print("uh oh")
        new_E_array[:,i ] = E_array[:,i]
    




    return E_array

In [ ]:
def declutter_E_array(E_array, dipoles,  r_data, discontinuity_threshold_std = 2,  energy_diff_threshold_std_p1 = 1,  E_disc_threshold = 3, energy_diff_threshold_std = 2,  num_to_declutter = 2):
    
    E_array_copy = np.array(E_array, copy = True)

    d_matrices = np.zeros((num_to_declutter , num_to_declutter, r_data.shape[0]) )
    diff_d_matrices =  np.zeros((num_to_declutter , num_to_declutter, r_data.shape[0]-1))

    def build_dipole_mag_matrix(dipoles, n_elec, r_):

        
        d_matrix = np.zeros((n_elec,n_elec))

        def vector_magnitude(vector):
            return np.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)

        for i in range(n_elec):
            for j in range(n_elec):
                d_matrix[i][j] = vector_magnitude(dipoles[i,j,:,r_])


        #d_matrix = d_matrix + d_matrix.T - np.diag(np.diag(d_matrix))

        return d_matrix
    
    
    
    for i in range(0, r_data.shape[0] ):
        d_matrices[:, : , i] = build_dipole_mag_matrix(dipoles, num_to_declutter, i)

    diff_d_matrices = np.diff(np.diff(d_matrices))
    diff_d_matrices[diff_d_matrices < 10**-12] = 0



    #store discontinyutity locs as a list of lists like  
    # [   [loc, surface_1]  ,     [loc, surface_1] , [loc, surface_1] ]
    #find all discontinutiniutiy locations based on continutity of transition dipoles and dipoles
    discontinuity_locs = []


    for z in range(0, 1):
        previous_intersection = 0 
        for i in range(0,num_to_declutter): 

            for j in range(i, num_to_declutter):
                dipoles_diff = diff_d_matrices[i,j, previous_intersection:]
                mean = np.mean(diff_d_matrices[i, j, previous_intersection:])

                std = np.std(diff_d_matrices[i, j, previous_intersection:])

                discontinuity_threshold_pos =  mean + (std*discontinuity_threshold_std)
                discontinuity_threshold_neg =  mean - (std*discontinuity_threshold_std)
                idx_1 = np.sort(np.concatenate([ np.where( dipoles_diff >  discontinuity_threshold_pos )[0]+1 ,  np.where( dipoles_diff <  discontinuity_threshold_neg )[0]+1 ]))
                #print(idx_1)

                #plt.plot(diff_d_matrices[i, j, previous_intersection:])

                if len(idx_1) != 0:
                    #getting first point of crossover
                    if len(idx_1) > 1:
                        idx_1_copy = idx_1.copy()
                        while(len(idx_1_copy) > 0 ):
                            #print(idx_1_copy)
                            crossover_loc = None
                            for q in range(len(idx_1_copy) - 1):
                                #print(idx_1_copy)
                                if idx_1_copy[q] +1 == idx_1_copy[q+1]:
                                    #print("hello")
                                    idx_1_copy = idx_1_copy[q+1:]
                                    pass
                                else:
                                    crossover_loc = idx_1_copy[q]

                                    discontinuity_locs.append([crossover_loc, j])
                                    try:
                                        idx_1_copy = idx_1_copy[q+1:]
                                    except():
                                        idx_1_copy = []
                                        break

                                    break

                                if len(idx_1_copy == 1):
                                    break

                            if len(idx_1_copy) == 1:    
                                idx_1_copy = []
                                discontinuity_locs.append([idx_1[-1], j])
                                break                
                        
                    elif len(idx_1) == 1:
                        discontinuity_locs.append([idx_1[0], j])


    #sort and remove repaets
    discontinuity_locs  = [list(x) for x in set(tuple(x) for x in discontinuity_locs )]
    discontinuity_locs = sorted(discontinuity_locs, key=lambda x: x[0])
    #print(discontinuity_locs)

    #store discontinity locations like this
    # [   [loc, surface_1, surface_2]  ,     [loc, surface_1, surface_2] , [loc, surface_1, surface_2] ]
    #find all discontinutiniutiy locations based on continutity of transition dipoles and dipoles

    new_discontinuity_locs = []
    for q in range(len(discontinuity_locs)):
        loc1 = discontinuity_locs[q]
        for w in range(len(discontinuity_locs)):
            if w != q:
                loc2 = discontinuity_locs[w]

                if loc1[1] != loc2[1]:

                    if loc1[0] == loc2[0]:
                        new_discontinuity_locs.append([loc1[0] ,discontinuity_locs[q][1], discontinuity_locs[w][1]])

                    if loc1[0] == loc2[0] -1:
                        new_discontinuity_locs.append([loc2[0] ,discontinuity_locs[q][1], discontinuity_locs[w][1]])

                    if loc1[0] -1 == loc2[0]:
                        new_discontinuity_locs.append([loc1[0] ,discontinuity_locs[q][1], discontinuity_locs[w][1]])

    #sort and remove repeats again
    discontinuity_locs = new_discontinuity_locs
    # we need to remove one of these [loc, 1,2] and [loc, 2,1]
    for i in range(len(discontinuity_locs)):
        if discontinuity_locs[i][1] > discontinuity_locs[i][2]:

            copy = discontinuity_locs[i][2] 
            discontinuity_locs[i][2] = discontinuity_locs[i][1] 
            discontinuity_locs[i][1] = copy

    discontinuity_locs  = [list(x) for x in set(tuple(x) for x in discontinuity_locs )]
    discontinuity_locs = sorted(discontinuity_locs, key=lambda x: x[0])


    #for every discontinuity loc check and see if energies are very close using standard deviations of differences
    #crossover energy arrays
    
    crossover_points = []
    for i in range(len(discontinuity_locs)):
        loc = discontinuity_locs[i]

        array1 = E_array[:, loc[1]]
        array2 = E_array[:, loc[2]]


        #trying to determine how close two energy surfaces get, if they get very close this some evidence that they crossover
        diff_array1 = np.diff(array1 )
        diff_array2 = np.diff(array2 )
        std1 = np.std(np.abs(diff_array1))
        mean1 = np.mean(np.abs(diff_array1))
        std2 = np.std(np.abs(diff_array2))
        mean2 = np.mean(np.abs(diff_array2))
        energy_diff_threshold = ((mean1 + mean2)/2) + (((std1+std2)/2) * energy_diff_threshold_std_p1)
        #find closest points
        #print(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])))
        closest_indices =np.where(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])) < energy_diff_threshold)[0]
        if loc[0] in closest_indices:

            crossover_points.append(loc)

        
    # a few points are like this :[1084, 3, 4] ,[1085, 3, 4], choose the larger one
    indices_to_pop = []
    for i in range(len(crossover_points)-1):
        if crossover_points[i][1] == crossover_points[i+1][1] and crossover_points[i][2] == crossover_points[i+1][2]:
            if crossover_points[i][0] +1 == crossover_points[i+1][0]:
                indices_to_pop.append(i)
    indices_to_pop.sort()
    indices_to_pop.reverse()
    for i in indices_to_pop:
        crossover_points.pop(i)


    #perform crossovers:
    for i in range(len(crossover_points)):


        idx = crossover_points[i][0] + 1

        array1 = E_array[:, crossover_points[i][1]]
        array2 = E_array[:, crossover_points[i][2]]


        

        array1_copy = np.array(array1, copy=True)
        array1 = np.concatenate([array1[:idx],  array2[idx:]])
        array2 =np.concatenate([array2[:idx] , array1_copy[idx:]])

        fitting_distance = 5
        array1 = array1.tolist()
        array2 = array2.tolist()
        r_data_list = r_data.tolist()
        #fitting region
        end_discontinuity = crossover_points[i][0]+1
        start_discontinuity = crossover_points[i][0] -1
        fit_E_data_end = min(end_discontinuity+fitting_distance, len(array1))
        fit_E_data_start= max(start_discontinuity-fitting_distance, 0)
        # print(fit_E_data_start)
        # print(fit_E_data_end)
        fitting_E_data = array1[fit_E_data_start: start_discontinuity] + array1[end_discontinuity: fit_E_data_end]
        fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
        r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
        polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
        array1 = array1[:fit_E_data_start] + polyvals.tolist() + array1[fit_E_data_end:]
        fitting_E_data = array2[fit_E_data_start: start_discontinuity] + array2[end_discontinuity: fit_E_data_end]
        fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
        r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
        polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
        array2 = array2[:fit_E_data_start] + polyvals.tolist() + array2[fit_E_data_end:]


        E_array[:,crossover_points[i][1]] = array1
        E_array[:,crossover_points[i][2]] = array2


        #go in and change crossover points to reflect changes in E_array
        surface_1 = crossover_points[i][1]
        surface_2 = crossover_points[i][2] 




        for p in range(len(crossover_points)):

            if crossover_points[p][1] == surface_1:
                crossover_points[p][1] = surface_2

            elif crossover_points[p][1] == surface_2:
                crossover_points[p][1] = surface_1

            if crossover_points[p][2] == surface_1:
                crossover_points[p][2] = surface_2

            elif crossover_points[p][2] == surface_2:
                crossover_points[p][2] = surface_1


    discontinuity_threshold_std = E_disc_threshold
    
    #dipole array orderd as [numroots][numroots][dipole vector][bondlength]
    E_array = np.copy(E_array)
    new_E_array = np.zeros_like(E_array)



    for i in range(0,num_to_declutter):
        previous_intersection = 0
        for z in range(0,50):
            for j in range(i+1, E_array.shape[1]):
                array1 = E_array[:, i]
                array2 = E_array[:, j]
                #Only want array from previous intersection so it doesnt get recrossed
                array1_from_previous_intersection = array1[previous_intersection:]
                array2_from_previous_intersection = array2[previous_intersection:]
                #trying to determine how close two energy surfaces get, if they get very close this some evidence that they crossover
                diff_array1 = np.diff(array1_from_previous_intersection)
                diff_array2 = np.diff(array2_from_previous_intersection)
                std1 = np.std(np.abs(diff_array1))
                mean1 = np.mean(np.abs(diff_array1))
                std2 = np.std(np.abs(diff_array2))
                mean2 = np.mean(np.abs(diff_array2))
                energy_diff_threshold = ((mean1 + mean2)/2) + (((std1+std2)/2) * energy_diff_threshold_std)
                #find closest points
                #print(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])))
                closest_indices =np.where(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])) < energy_diff_threshold)
                try:
                    #use discontinuties in second derivative, discontinutities defined using standard deviation
                    dy_1 = np.abs(np.gradient(np.gradient(array1_from_previous_intersection, r_data[previous_intersection:], edge_order = 1), r_data[previous_intersection:], edge_order = 1))
                    std = np.std(abs(np.diff(dy_1)))
                    mean = np.mean(abs(np.diff(dy_1)))
                    discontinuity_threshold =  mean + (std*discontinuity_threshold_std)
                    idx_1 = np.where(abs(np.diff(dy_1)) >  discontinuity_threshold)[0]+2
                    dy_2= np.abs(np.gradient(np.gradient(array2_from_previous_intersection, r_data[previous_intersection:], edge_order=1), r_data[previous_intersection:], edge_order=1))
                    std = np.std(abs(np.diff(dy_2)))
                    mean = np.mean(abs(np.diff(dy_2)))
                    discontinuity_threshold =  mean + (std*discontinuity_threshold_std)
                    idx_2 = np.where(abs(np.diff(dy_2)) > discontinuity_threshold)[0]+2
                    if (len(idx_1)!= 0 and len(idx_2) != 0 ):
                        mask_idx1_idx2 = np.isin(idx_1, idx_2)
                        indices_idx1_in_idx2 = np.where(mask_idx1_idx2)[0]
                        indices_idx1_in_idx2 = idx_1[indices_idx1_in_idx2]
                        # indices_idx1_in_idx2  = indices_idx1_in_idx2[ending_index:]
                        # starting_index=ending_index
                        # ending_index = starting_index
                        starting_index = 0
                        ending_index = 0
                        for elem_index in range(len(indices_idx1_in_idx2)-1):
                            #print("ayo: ", abs((indices_idx1_in_idx2[elem_index]) - (indices_idx1_in_idx2[elem_index+1])))
                            if abs((indices_idx1_in_idx2[elem_index]) - (indices_idx1_in_idx2[elem_index+1])) < 25 :
                                ending_index = ending_index+1
                            else:
                                break
                        indices_idx1_in_idx2 = indices_idx1_in_idx2[starting_index:ending_index]
                        if(len(indices_idx1_in_idx2) != 0 ):
                            mask_discontinuties_energydiff = np.isin(indices_idx1_in_idx2, closest_indices)
                            indices_discontinuties_in_energydiff = np.where(mask_discontinuties_energydiff)[0]
                            #print(indices_discontinuties_in_energydiff)
                            #print(indices_discontinuties_in_energydiff)
                            if len(indices_discontinuties_in_energydiff) != 0 :
                                for k in range(len(indices_discontinuties_in_energydiff) - 1):
                                    idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[k]]+ previous_intersection
                                    #print(idx)
                                    array1_copy = np.array(array1, copy=True)
                                    array1 = np.concatenate([array1[:idx],  array2[idx:idx+1], array1[idx+1:]])
                                    array2 = np.concatenate([array2[:idx] , array1_copy[idx:idx+1], array2[idx+1:]])
                                    E_array[:,i] = array1
                                    E_array[:,j] = array2
                                idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[-1]]+ previous_intersection
                                #print(idx)
                                array1_copy = np.array(array1, copy=True)
                                array1 = np.concatenate([array1[:idx],  array2[idx:]])
                                array2 =np.concatenate([array2[:idx] , array1_copy[idx:]])
                                #print(indices_idx1_in_idx2)
                                fitting_distance=25

                                if True : #abs(indices_idx1_in_idx2[-1] - indices_idx1_in_idx2[0]) < fitting_distance:
                                    array1 = array1.tolist()
                                    array2 = array2.tolist()
                                    r_data_list = r_data.tolist()
                                    #fitting region
                                    end_discontinuity = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[-1]]+ previous_intersection
                                    start_discontinuity = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[0]]+ previous_intersection
                                    fit_E_data_end = min(end_discontinuity+fitting_distance, len(array1))
                                    fit_E_data_start= max(start_discontinuity-fitting_distance, 0)
                                    # print(fit_E_data_start)
                                    # print(fit_E_data_end)
                                    fitting_E_data = array1[fit_E_data_start: start_discontinuity] + array1[end_discontinuity: fit_E_data_end]
                                    fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
                                    with warnings.catch_warnings():
                                        warnings.simplefilter("ignore")
                                        poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
                                    r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
                                    polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
                                    array1 = array1[:fit_E_data_start] + polyvals.tolist() + array1[fit_E_data_end:]
                                    fitting_E_data = array2[fit_E_data_start: start_discontinuity] + array2[end_discontinuity: fit_E_data_end]
                                    fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
                                    with warnings.catch_warnings():
                                        warnings.simplefilter("ignore")
                                        poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
                                    r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
                                    polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
                                    array2 = array2[:fit_E_data_start] + polyvals.tolist() + array2[fit_E_data_end:]
                                E_array[:,i] = array1
                                E_array[:,j] = array2
                                previous_intersection = idx
                except():
                    print("uh oh")
        new_E_array[:,i ] = E_array[:,i]
    




    return E_array

In [ ]:
# array_data = "/Users/proden/Code/SCQED-PCQED/array_data"
# cavity_E_array_0_05_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_05.npy").T [20:, :]
# r_data = np.load(array_data + "/MgH/fci_r_array_MgH.npy")[20:]
# dipoles_05 = np.load(array_data + "/MgH/dipoles_MgH0_05.npy")[:,:,:,20:]
# N_R = r_data.shape[0]
# N_R = r_data.shape[0]
# #N_R = 200
# r_data = np.linspace(0.5, 3.0, N_R)


# print(cavity_E_array_0_05_T[:, 3])
# print(cavity_E_array_0_05_T[:, 4])

In [ ]:
# array_data = "/Users/proden/Code/SCQED-PCQED/array_data"
# cavity_E_array_0_05_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_05.npy").T [20:, :]
# r_data = np.load(array_data + "/MgH/fci_r_array_MgH.npy")[20:]
# dipoles_05 = np.load(array_data + "/MgH/dipoles_MgH0_05.npy")[:,:,:,20:]
# N_R = r_data.shape[0]
# N_R = r_data.shape[0]
# #N_R = 200
# r_data = np.linspace(0.5, 3.0, N_R)

# plt.plot(cavity_E_array_0_05_T)
# plt.show()
# #cavity_E_array_0_05  = declutter_E_array(cavity_E_array_0_05_T, dipoles_05, r_data, 1 , 0.1 , 1,  0.2 ,num_to_declutter=10)
# for i in range(0,5):
#     cavity_E_array_0_05  = declutter_E_array(cavity_E_array_0_05_T, dipoles_05, r_data, 800, 1 , 1,  0.1 ,num_to_declutter=10)
# plt.plot(cavity_E_array_0_05)
# plt.show()

In [ ]:
# array_data = "C:/Users/peyto/Documents/schol/code/SCQED-PCQED/array_data"
# cavity_E_array_0_01_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_01.npy").T [20:, :]
# r_data = np.load(array_data + "/MgH/fci_r_array_MgH.npy")[20:]
# dipoles_01 = np.load(array_data + "/MgH/dipoles_MgH0.npy")[:,:,:,20:]
# N_R = r_data.shape[0]
# N_R = r_data.shape[0]
# #N_R = 200
# r_data = np.linspace(0.5, 3.0, N_R)

# plt.plot(cavity_E_array_0_01_T)
# plt.show()
# cavity_E_array_0_01  = declutter_E_array(cavity_E_array_0_01_T, dipoles_01, r_data,1, 0.1 , 2,  0.5 ,num_to_declutter=10)
# plt.plot(cavity_E_array_0_01)
# plt.show()

In [ ]:
# array_data = "C:/Users/peyto/Documents/schol/code/SCQED-PCQED/array_data"
# cavity_E_array_0_00_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0.npy").T [20:, :]
# r_data = np.load(array_data + "/MgH/fci_r_array_MgH.npy")[20:]
# dipoles_0 = np.load(array_data + "/MgH/dipoles_MgH0.npy")[:,:,:,20:]
# N_R = r_data.shape[0]
# N_R = r_data.shape[0]
# #N_R = 200
# r_data = np.linspace(0.5, 3.0, N_R)

# plt.plot(cavity_E_array_0_00_T)
# plt.show()
# #cavity_E_array_0_00  = declutter_E_array(cavity_E_array_0_00_T, dipoles_0, r_data,5, 0.1 , 2,  1,num_to_declutter=10)
# cavity_E_array_0_00  = declutter_E_array(cavity_E_array_0_00_T, dipoles_0, r_data, 1, 0.1 , 2,  0.5 ,num_to_declutter=10)
# plt.plot(cavity_E_array_0_00)
# plt.show()

In [ ]:
plt.plot(cavity_E_array_0_00_T)
plt.show()
plt.plot(cavity_E_array_0_001_T)
plt.show()
plt.plot(cavity_E_array_0_01_T)
plt.show()
plt.plot(cavity_E_array_0_02_T)
plt.show()
plt.plot(cavity_E_array_0_03_T)
plt.show()
plt.plot(cavity_E_array_0_04_T)
plt.show()

In [ ]:

cavity_E_array_0_00  = declutter_E_array(cavity_E_array_0_00_T, dipoles_0, r_data, 1, 0.1 , 2,  0.5,num_to_declutter=10)
cavity_E_array_0_001  = declutter_E_array(cavity_E_array_0_001_T, dipoles_001, r_data, 8, 1, 9,  1,num_to_declutter=10)
cavity_E_array_0_005  = declutter_E_array(cavity_E_array_0_005_T, dipoles_005, r_data,7, 1.2 , 1,  0.2,num_to_declutter=10)
cavity_E_array_0_01  = declutter_E_array(cavity_E_array_0_01_T, dipoles_01, r_data, 7, 1.2 , 1,  0.2,num_to_declutter=10)
cavity_E_array_0_02  = declutter_E_array(cavity_E_array_0_02_T, dipoles_02, r_data, 8,1, 3,  1,num_to_declutter=10)
cavity_E_array_0_03  = declutter_E_array(cavity_E_array_0_03_T, dipoles_03, r_data, 8,1,3, 1,num_to_declutter=10)
cavity_E_array_0_04  = declutter_E_array(cavity_E_array_0_04_T, dipoles_04, r_data,8,1,3, 1,num_to_declutter=10)
cavity_E_array_0_05  = declutter_E_array(cavity_E_array_0_05_T, dipoles_05, r_data, 8,1,3, 1,num_to_declutter=10)


In [ ]:
plt.plot(cavity_E_array_0_00)
plt.show()
plt.plot(cavity_E_array_0_001)
plt.show()
plt.plot(cavity_E_array_0_005)
plt.show()
plt.plot(cavity_E_array_0_01)
plt.show()
plt.plot(cavity_E_array_0_02)
plt.show()
plt.plot(cavity_E_array_0_03)
plt.show()
plt.plot(cavity_E_array_0_04)
plt.show()
plt.plot(cavity_E_array_0_05)
plt.show()

In [ ]:

LP_0_001 =           cavity_E_array_0_001[:,2]
LP_0_005 =            cavity_E_array_0_005[:,2]
LP_0_01 =             cavity_E_array_0_01[:,2]
LP_0_02 =             cavity_E_array_0_02[:,2]
LP_0_03 =             cavity_E_array_0_03[:,2]
LP_0_04 =             cavity_E_array_0_04[:,2]
LP_0_05 =             cavity_E_array_0_05[:,2]
              


UP_0_00 =             cavity_E_array_0_00[:,3]
UP_0_001 =           cavity_E_array_0_001[:,3]
UP_0_005 =            cavity_E_array_0_005[:,3]
UP_0_01 =             cavity_E_array_0_01[:,3]
UP_0_02 =             cavity_E_array_0_02[:,3]
UP_0_03 =             cavity_E_array_0_03[:,3]
UP_0_04 =             cavity_E_array_0_04[:,3]
UP_0_05 =             cavity_E_array_0_05[:,3]
              

S0_array = cavity_E_array_0_00[:,0]
S1_array = cavity_E_array_0_00[:,3]


plt.plot(LP_0_001)
plt.plot(LP_0_02)
plt.plot(LP_0_03)
plt.plot(UP_0_001)
plt.plot(UP_0_02)
plt.plot(UP_0_03)
plt.show()

In [ ]:
import warnings
from scipy.signal import find_peaks
def declutter_E_array(E_array, dipoles,  r_data, discontinuity_threshold_std = 1, energy_diff_threshold_std = 2,  num_to_declutter = 2):


    #dipole array orderd as [numroots][numroots][dipole vector][bondlength]
    E_array = np.copy(E_array)
    new_E_array = np.zeros_like(E_array)


    d_reshaped= np.zeros_like(E_array)

    def vector_magnitude(vector):
        return np.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)


    for q in range(dipoles.shape[3]):
        for i in range(dipoles.shape[0]):
            for j in range(dipoles.shape[1]):
                if i == j:
                    d_reshaped[q][i] = vector_magnitude(dipoles[i,i,:,q])
  
    dipoles = np.copy(d_reshaped)
    new_dipoles = np.zeros_like(dipoles)

    # plt.plot(dipoles)
    # plt.show()



    for q in range(0, 5):
        #dipole discontinuities
        for i in range(0,num_to_declutter):
            previous_intersection = 0
            for z in range(0,50):
                for j in range(i+1, E_array.shape[1]):
                    array1 = E_array[:, i]
                    array2 = E_array[:, j]

                    dipole_array1 = dipoles[:, i] [previous_intersection:]
                    dipole_array2 = dipoles[:, j] [previous_intersection:]


                    #Only want array from previous intersection so it doesnt get recrossed
                    array1_from_previous_intersection = array1[previous_intersection:]
                    array2_from_previous_intersection = array2[previous_intersection:]
                    #trying to determine how close two energy surfaces get, if they get very close this some evidence that they crossover
                    diff_array1 = np.diff(array1_from_previous_intersection)
                    diff_array2 = np.diff(array2_from_previous_intersection)
                    std1 = np.std(np.abs(diff_array1))
                    mean1 = np.mean(np.abs(diff_array1))
                    std2 = np.std(np.abs(diff_array2))
                    mean2 = np.mean(np.abs(diff_array2))
                    energy_diff_threshold = ((mean1 + mean2)/2) + (((std1+std2)/2) * energy_diff_threshold_std)
                    #find closest points
                    #print(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])))
                    closest_indices =np.where(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])) < energy_diff_threshold)
                    try:

                        #if energies are close and there are two matcjhing discontinuities in dipole arrary 
                        dipoles_diff_1 = np.diff(dipole_array1)
                        dipoles_diff_2 = np.diff(dipole_array2)

                        mean_1 = np.mean(dipoles_diff_1)
                        mean_2 = np.mean(dipoles_diff_2)

                        std_1 = np.std(dipoles_diff_1)
                        std_2 = np.std(dipoles_diff_2)
                        discontinuity_threshold_pos =  mean_1 + (std_1*discontinuity_threshold_std)
                        discontinuity_threshold_neg =  mean_1 - (std_1*discontinuity_threshold_std)
                        idx_1 = np.sort(np.concatenate([ np.where( dipoles_diff_1 >  discontinuity_threshold_pos )[0]+1 ,  np.where( dipoles_diff_1 <  discontinuity_threshold_neg )[0]+1 ]))

                        discontinuity_threshold_pos =  mean_2 + (std_2*discontinuity_threshold_std)
                        discontinuity_threshold_neg =  mean_2 - (std_2*discontinuity_threshold_std)
                        idx_2 = np.sort(np.concatenate([ np.where( dipoles_diff_2 >  discontinuity_threshold_pos )[0]+1 ,  np.where( dipoles_diff_2 <  discontinuity_threshold_neg )[0]+1 ]))



                        peaks_1 = find_peaks( np.abs(np.diff(dipoles_diff_1)) , prominence= np.mean(np.abs(np.diff(dipoles_diff_1))) )
                        peaks_2 = find_peaks( np.abs(np.diff(dipoles_diff_2)) , prominence= np.mean(np.abs(np.diff(dipoles_diff_1))) )


                        
                        peaks_1 = find_peaks( np.abs(dipoles_diff_1) , prominence= np.mean(np.abs(dipoles_diff_1)) )
                        peaks_2 = find_peaks( np.abs(dipoles_diff_2) , prominence= np.mean(np.abs(dipoles_diff_1)) )



                        idx_1 = peaks_1[0]+1
                        idx_2 = peaks_2[0]+1


                        if (len(idx_1)!= 0 and len(idx_2) != 0 ):
                            mask_idx1_idx2 = np.isin(idx_1, idx_2)
                            indices_idx1_in_idx2 = np.where(mask_idx1_idx2)[0]
                            indices_idx1_in_idx2 = idx_1[indices_idx1_in_idx2]

                            
                            if(len(indices_idx1_in_idx2) != 0 ):
                                mask_discontinuties_energydiff = np.isin(indices_idx1_in_idx2, closest_indices)
                                indices_discontinuties_in_energydiff = np.where(mask_discontinuties_energydiff)[0]

                                if len(indices_discontinuties_in_energydiff) != 0 :

                                    idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[0]]+ previous_intersection
                                    #print(idx)
                                    array1_copy = np.array(array1, copy=True)
                                    array1 = np.concatenate([array1[:idx],  array2[idx:]])
                                    array2 =np.concatenate([array2[:idx] , array1_copy[idx:]])


                                    dipole_array1 = dipoles[:, i] 
                                    dipole_array2 = dipoles[:, j]


                                    dipole_array1_copy = np.array(dipole_array1, copy=True)
                                    dipole_array1 = np.concatenate([dipole_array1[:idx],  dipole_array2[idx:]])
                                    dipole_array2 =np.concatenate([dipole_array2[:idx] , dipole_array1_copy[idx:]])

                                    dipoles[:,i] = dipole_array1
                                    dipoles[:,j] = dipole_array2
                                    E_array[:,i] = array1
                                    E_array[:,j] = array2
                                    previous_intersection = idx+1
                    except():
                        print("uh oh")
            new_E_array[:,i ] = E_array[:,i]
            new_dipoles[:,i ] = dipoles[:, i]


            # plt.plot(new_dipoles)
            # plt.show()

        E_array = np.copy(new_E_array)
        new_E_array = np.zeros_like(E_array)


    for i in range(0,num_to_declutter):
        previous_intersection = 0
        for z in range(0,50):
            for j in range(i+1, E_array.shape[1]):
                array1 = E_array[:, i]
                array2 = E_array[:, j]
                #Only want array from previous intersection so it doesnt get recrossed
                array1_from_previous_intersection = array1[previous_intersection:]
                array2_from_previous_intersection = array2[previous_intersection:]
                #trying to determine how close two energy surfaces get, if they get very close this some evidence that they crossover
                diff_array1 = np.diff(array1_from_previous_intersection)
                diff_array2 = np.diff(array2_from_previous_intersection)
                std1 = np.std(np.abs(diff_array1))
                mean1 = np.mean(np.abs(diff_array1))
                std2 = np.std(np.abs(diff_array2))
                mean2 = np.mean(np.abs(diff_array2))
                energy_diff_threshold = ((mean1 + mean2)/2) + (((std1+std2)/2) * energy_diff_threshold_std)
                #find closest points
                #print(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])))
                closest_indices =np.where(np.abs(np.abs(array1[previous_intersection:]) - np.abs(array2[previous_intersection:])) < energy_diff_threshold)
                try:
                    #use discontinuties in second derivative, discontinutities defined using standard deviation
                    dy_1 = np.abs(np.gradient(np.gradient(array1_from_previous_intersection, r_data[previous_intersection:], edge_order = 1), r_data[previous_intersection:], edge_order = 1))
                    std = np.std(abs(np.diff(dy_1)))
                    mean = np.mean(abs(np.diff(dy_1)))
                    discontinuity_threshold =  mean + (std*discontinuity_threshold_std)
                    idx_1 = np.where(abs(np.diff(dy_1)) >  discontinuity_threshold)[0]+2
                    dy_2= np.abs(np.gradient(np.gradient(array2_from_previous_intersection, r_data[previous_intersection:], edge_order=1), r_data[previous_intersection:], edge_order=1))
                    std = np.std(abs(np.diff(dy_2)))
                    mean = np.mean(abs(np.diff(dy_2)))
                    discontinuity_threshold =  mean + (std*discontinuity_threshold_std)
                    idx_2 = np.where(abs(np.diff(dy_2)) > discontinuity_threshold)[0]+2
                    if (len(idx_1)!= 0 and len(idx_2) != 0 ):
                        mask_idx1_idx2 = np.isin(idx_1, idx_2)
                        indices_idx1_in_idx2 = np.where(mask_idx1_idx2)[0]
                        indices_idx1_in_idx2 = idx_1[indices_idx1_in_idx2]
                        # indices_idx1_in_idx2  = indices_idx1_in_idx2[ending_index:]
                        # starting_index=ending_index
                        # ending_index = starting_index
                        starting_index = 0
                        ending_index = 0
                        for elem_index in range(len(indices_idx1_in_idx2)-1):
                            #print("ayo: ", abs((indices_idx1_in_idx2[elem_index]) - (indices_idx1_in_idx2[elem_index+1])))
                            if abs((indices_idx1_in_idx2[elem_index]) - (indices_idx1_in_idx2[elem_index+1])) < 25 :
                                ending_index = ending_index+1
                            else:
                                break
                        indices_idx1_in_idx2 = indices_idx1_in_idx2[starting_index:ending_index]
                        if(len(indices_idx1_in_idx2) != 0 ):
                            mask_discontinuties_energydiff = np.isin(indices_idx1_in_idx2, closest_indices)
                            indices_discontinuties_in_energydiff = np.where(mask_discontinuties_energydiff)[0]
                            #print(indices_discontinuties_in_energydiff)
                            #print(indices_discontinuties_in_energydiff)
                            if len(indices_discontinuties_in_energydiff) != 0 :
                                for k in range(len(indices_discontinuties_in_energydiff) - 1):
                                    idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[k]]+ previous_intersection
                                    #print(idx)
                                    array1_copy = np.array(array1, copy=True)
                                    array1 = np.concatenate([array1[:idx],  array2[idx:idx+1], array1[idx+1:]])
                                    array2 = np.concatenate([array2[:idx] , array1_copy[idx:idx+1], array2[idx+1:]])
                                    E_array[:,i] = array1
                                    E_array[:,j] = array2
                                idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[-1]]+ previous_intersection
                                #print(idx)
                                array1_copy = np.array(array1, copy=True)
                                array1 = np.concatenate([array1[:idx],  array2[idx:]])
                                array2 =np.concatenate([array2[:idx] , array1_copy[idx:]])
                                #print(indices_idx1_in_idx2)
                                fitting_distance=10
                                if abs(indices_idx1_in_idx2[-1] - indices_idx1_in_idx2[0]) < fitting_distance:
                                    array1 = array1.tolist()
                                    array2 = array2.tolist()
                                    r_data_list = r_data.tolist()
                                    #fitting region
                                    end_discontinuity = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[-1]]+ previous_intersection
                                    start_discontinuity = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[0]]+ previous_intersection
                                    fit_E_data_end = min(end_discontinuity+fitting_distance, len(array1))
                                    fit_E_data_start= max(start_discontinuity-fitting_distance, 0)
                                    # print(fit_E_data_start)
                                    # print(fit_E_data_end)
                                    fitting_E_data = array1[fit_E_data_start: start_discontinuity] + array1[end_discontinuity: fit_E_data_end]
                                    fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
                                    with warnings.catch_warnings():
                                        warnings.simplefilter("ignore")
                                        poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
                                    r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
                                    polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
                                    array1 = array1[:fit_E_data_start] + polyvals.tolist() + array1[fit_E_data_end:]
                                    fitting_E_data = array2[fit_E_data_start: start_discontinuity] + array2[end_discontinuity: fit_E_data_end]
                                    fitting_r_data = r_data_list[fit_E_data_start: start_discontinuity] + r_data_list[end_discontinuity:fit_E_data_end]
                                    with warnings.catch_warnings():
                                        warnings.simplefilter("ignore")
                                        poly = np.poly1d(np.polyfit(fitting_r_data, fitting_E_data, 12))
                                    r_data_fitting_list =  r_data_list[fit_E_data_start:fit_E_data_end]
                                    polyvals = np.polyval(np.asarray(poly),r_data_fitting_list )
                                    array2 = array2[:fit_E_data_start] + polyvals.tolist() + array2[fit_E_data_end:]
                                E_array[:,i] = array1
                                E_array[:,j] = array2
                                previous_intersection = idx
                except():
                    print("uh oh")
        new_E_array[:,i ] = E_array[:,i]
    



    return new_E_array

In [ ]:
array_data = "/Users/proden/Code/SCQED-PCQED/array_data"
cavity_E_array_0_00_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0.npy").T [20:, :]
cavity_E_array_0_001_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_001.npy").T [20:, :]
cavity_E_array_0_005_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_005.npy").T[20:, :]
cavity_E_array_0_01_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_01.npy").T[20:, :]
cavity_E_array_0_02_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_02.npy").T[20:, :]
cavity_E_array_0_03_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_03.npy").T[20:, :]
cavity_E_array_0_04_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_04.npy").T[20:, :]
cavity_E_array_0_05_T = np.load(array_data + "/MgH/fci_cavity_arrays_MgH_631G0_05.npy").T[20:, :]
r_data = np.load(array_data + "/MgH/fci_r_array_MgH.npy")[20:]
dipoles_0 = np.load(array_data + "/MgH/dipoles_MgH0.npy")[:,:,:,20:]
dipoles_001 = np.load(array_data + "/MgH/dipoles_MgH0_001.npy")[:,:,:,20:]
dipoles_005 = np.load(array_data + "/MgH/dipoles_MgH0_005.npy")[:,:,:,20:]
dipoles_01 = np.load(array_data + "/MgH/dipoles_MgH0_01.npy")[:,:,:,20:]
dipoles_02 = np.load(array_data + "/MgH/dipoles_MgH0_02.npy")[:,:,:,20:]
dipoles_03 = np.load(array_data + "/MgH/dipoles_MgH0_03.npy")[:,:,:,20:]
dipoles_04 = np.load(array_data + "/MgH/dipoles_MgH0_04.npy")[:,:,:,20:]
dipoles_05 = np.load(array_data + "/MgH/dipoles_MgH0_05.npy")[:,:,:,20:]
r_data = np.load(array_data + "/MgH/fci_r_array_MgH.npy")[20:]
N_R = r_data.shape[0]
N_R = r_data.shape[0]
#N_R = 200
r_data = np.linspace(0.5, 3.0, N_R)

In [ ]:


cavity_E_array_0_00  = declutter_E_array(cavity_E_array_0_00_T, dipoles_0, r_data,4, -0.075,num_to_declutter=10)
cavity_E_array_0_001  = declutter_E_array(cavity_E_array_0_001_T, dipoles_001, r_data, 12,-0.075,num_to_declutter=10)
cavity_E_array_0_005  = declutter_E_array(cavity_E_array_0_005_T , dipoles_005, r_data,4,-0.075,num_to_declutter=10)
cavity_E_array_0_01  = declutter_E_array(cavity_E_array_0_01_T, dipoles_01, r_data, 4,-0.075,num_to_declutter=10)
cavity_E_array_0_02  = declutter_E_array(cavity_E_array_0_02_T, dipoles_02, r_data,4,-0.075,num_to_declutter=10)
cavity_E_array_0_03  = declutter_E_array(cavity_E_array_0_03_T, dipoles_03, r_data,4,-0.075,num_to_declutter=10)
cavity_E_array_0_04  = declutter_E_array(cavity_E_array_0_04_T, dipoles_04, r_data,4,0.1,num_to_declutter=10)
cavity_E_array_0_05  = declutter_E_array(cavity_E_array_0_05_T, dipoles_05, r_data, 4,-0.075,num_to_declutter=10)

In [ ]:
plt.plot(cavity_E_array_0_00)
plt.show()
plt.plot(cavity_E_array_0_001)
plt.show()
plt.plot(cavity_E_array_0_005)
plt.show()
plt.plot(cavity_E_array_0_01)
plt.show()
plt.plot(cavity_E_array_0_02)
plt.show()
plt.plot(cavity_E_array_0_03)
plt.show()
plt.plot(cavity_E_array_0_04)
plt.show()
plt.plot(cavity_E_array_0_05)
plt.show()

In [ ]:

LP_0_001 =           cavity_E_array_0_001[:,2]
LP_0_005 =            cavity_E_array_0_005[:,2]
LP_0_01 =             cavity_E_array_0_01[:,2]
LP_0_02 =             cavity_E_array_0_02[:,2]
LP_0_03 =             cavity_E_array_0_03[:,2]
LP_0_04 =             cavity_E_array_0_04[:,2]
LP_0_05 =             cavity_E_array_0_05[:,2]
              


UP_0_00 =             cavity_E_array_0_00[:,3]
UP_0_001 =           cavity_E_array_0_001[:,3]
UP_0_005 =            cavity_E_array_0_005[:,3]
UP_0_01 =             cavity_E_array_0_01[:,3]
UP_0_02 =             cavity_E_array_0_02[:,3]
UP_0_03 =             cavity_E_array_0_03[:,3]
UP_0_04 =             cavity_E_array_0_04[:,3]
UP_0_05 =             cavity_E_array_0_05[:,3]
              

S0_array = cavity_E_array_0_00[:,0]
S1_array = cavity_E_array_0_00[:,3]


plt.plot(LP_0_001)
plt.plot(LP_0_005)
plt.plot(LP_0_02)
plt.plot(LP_0_03)
plt.plot(LP_0_04)
plt.plot(LP_0_05)
plt.plot(UP_0_001)
plt.plot(UP_0_005)
plt.plot(UP_0_02)
plt.plot(UP_0_03)
plt.plot(UP_0_04)
plt.plot(UP_0_05)
plt.show()

In [ ]:

hbar = 1

# number of grid points 
N = 2001


def get_fd_wfn(x, V_y: np.array, use_5_point_stencil = False):

    hbar = 1


    # define grid spacing h
    h = x[1]-x[0]

    # create arrays for T, V, and H - we truncate the smallest and largest grid points where 
    # the centered finite difference derivatives cannot be defined
    T = np.zeros((N-2, N-2))
    V = np.zeros((N-2, N-2))
    H = np.zeros((N-2, N-2))

    # this uses the 3 point stencil; we can adapt to use a 5 point and it might improve accuracy


    if not use_5_point_stencil:
        for i in range(N-2):
            for j in range(N-2):
                if i==j:
                    T[i,j]= -2
                elif np.abs(i-j)==1:
                    T[i,j]=1
                else:
                    T[i,j]=0

        T = -T *( hbar ** 2 / (2 * mu_au* h**2))
        #T =  (- (hbar ** 2) / (2* mu_kg)) *  (1 / ( h**2)) * joule_to_hartree  * T


    elif use_5_point_stencil:
        for i in range(N-2):
            for j in range(N-2):
                if i==j:
                    T[i,j]= -30
                elif np.abs(i-j)==1:
                    T[i,j]=16
                elif np.abs(i-j)==2:
                    T[i,j]=-1

        T = -T *  ((hbar ** 2) / (2* mu_au))*  (1 / ( 12 * h**2)) 


    for i in range(N-2):
        for j in range(N-2):
            if i==j:
                V[i,j]= V_y[i+1]
            else:
                V[i,j]=0
                
    H = T + V

    #print((-T * hbar ** 2 / (2 * mu_kg* h**2)) * (2.294 * 10 ** 17))
    #print(V)

    vals, vecs = np.linalg.eigh(H)

    if np.average(vecs[:, 0]) < 0:
        vecs = vecs * -1

    return vals, vecs

In [ ]:
#FCF calculator
def get_fcf_matrix(potential_1, potential_2, r_data, matrix_size = 6, potential_1_is_groundstate = False):

    #geneerate 2 sets of wavefunctions for the potentials

    r_data_au = r_data / psi4.constants.bohr2angstroms

    min_potential_1_loc = np.argmin(potential_1[:])
    r_eq_au =r_data_au[potential_1.argmin()]

    print("r_eq_au : " , r_eq_au)


    # Fitting S0 PES to a quintic polynomial

    poly = np.poly1d(np.polyfit(r_data_au, potential_1, 14))

    poly_array = np.asarray(poly)


    #Taking first and second derivative of S0 PES and evaluating at r_eq
    first_derivative = poly.deriv()
    second_derivative = first_derivative.deriv()
    k_au = second_derivative(r_eq_au)
    print("k_au: ", k_au)


    angstrom_to_bohr = 1.88973
    x_min = r_data_au[0]
    x_max = r_data_au[-1]

    hbar = 1

    # number of grid points 
    N = 2001
    # define grid
    x = np.linspace(x_min, x_max, N)

    V_y = np.polyval(np.asarray(poly), (x))


    vals1, vecs1 = get_fd_wfn(x, V_y, use_5_point_stencil=True)
    #vals1, vecs1 = get_fd_wfn(x, V_y)



    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('bond length (m)')
    ax1.set_ylabel('wfn', color=color)
    ax1.plot(x[1:N-1], vecs1[:,0], 'r', label = "$\psi_0$")
    ax1.plot(x[1:N-1], vecs1[:,1], 'b',label = "$\psi_1$" )
    ax1.plot(x[1:N-1], vecs1[:,2], 'g',label = "$\psi_2$")
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel('energy (hartree)', color=color)  # we already handled the x-label with ax1
    ax2.plot(r_data_au, potential_1, 'bo', label='PES_1')
    ax2.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    fig.legend()
    plt.show()


    min_potential_2_loc = np.argmin(potential_2[:])
    r_eq_au =r_data_au[min_potential_2_loc]

    print("r_eq_au : " , r_eq_au)


    # Fitting S0 PES to a quintic polynomial

    #can use this line to only fit to bottom of well for cubic and harmonic
    #poly = np.poly1d(np.polyfit(r_data_meters[50:100], fci_S0[50:100], 4))

    poly = np.poly1d(np.polyfit(r_data_au, potential_2, 14))

    poly_array = np.asarray(poly)


    #Taking first and second derivative of S0 PES and evaluating at r_eq
    first_derivative = poly.deriv()
    second_derivative = first_derivative.deriv()
    k_au = second_derivative(r_eq_au)
    print("k_au: ", k_au)


    angstrom_to_bohr = 1.88973
    x_min = r_data_au[0]
    x_max = r_data_au[-1]

    hbar = 1

    # number of grid points 
    N = 2001
    # define grid
    x = np.linspace(x_min, x_max, N)

    V_y = np.polyval(np.asarray(poly), (x))


    vals2, vecs2 = get_fd_wfn(x, V_y, use_5_point_stencil=True)
    #vals2, vecs2 = get_fd_wfn(x, V_y)



    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('bond length')
    ax1.set_ylabel('wfn', color=color)
    ax1.plot(x[1:N-1], vecs2[:,0], 'r',label = "$\psi_0$")
    ax1.plot(x[1:N-1], vecs2[:,1], 'b',label = "$\psi_1$")
    ax1.plot(x[1:N-1], vecs2[:,2], 'g', label = "$\psi_2$")

    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel('energy (hartree)', color=color)  # we already handled the x-label with ax1
    ax2.plot(r_data_au, potential_2, 'bo', label='PES_2')
    ax2.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    fig.legend()
    plt.show()


    FCF_matrix = np.zeros((matrix_size,matrix_size))

    for i in range(FCF_matrix.shape[0]):
        for j in range(FCF_matrix.shape[0]):

            FCF_matrix[i][j] = np.trapz(vecs1[:,i] * vecs2[:,j]) 
            FCF = np.absolute(FCF_matrix) ** 2 

    return FCF



In [ ]:

fcf_0_05 = get_fcf_matrix(S0_array,UP_0_05, r_data, 6)

fcf_0_04 = get_fcf_matrix(S0_array,UP_0_04, r_data, 6)

fcf_0_03 = get_fcf_matrix(S0_array,UP_0_03, r_data, 6)

fcf_0_02 = get_fcf_matrix(S0_array,UP_0_02, r_data, 6)

fcf_0_005 = get_fcf_matrix(S0_array,UP_0_005, r_data, 6)

fcf_0_001 = get_fcf_matrix(S0_array,UP_0_001, r_data, 6)

In [ ]:
min_val, max_val = 0, 15
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_0_001)
x=x.style.background_gradient(cmap=cm)
display(x)


min_val, max_val = 0, 15
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_0_005)
x=x.style.background_gradient(cmap=cm)
display(x)

min_val, max_val = 0, 15
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_0_02)
x=x.style.background_gradient(cmap=cm)
display(x)

min_val, max_val = 0, 15
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_0_03)
x=x.style.background_gradient(cmap=cm)
display(x)


min_val, max_val = 0, 15
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_0_04)
x=x.style.background_gradient(cmap=cm)
display(x)



min_val, max_val = 0, 15
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_0_05)
x=x.style.background_gradient(cmap=cm)
display(x)
